<a href="https://colab.research.google.com/github/StealStar/Auto/blob/Colab-code/%EC%9E%90%EB%8F%99%EC%82%AC%EB%83%A5_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-texttospeech  # TTS 라이브러리 설치
!pip install openai  # 또는 다른 AI API 라이브러리

In [ ]:
# 기존에 작성하신 코드 전체를 여기에 붙여넣기

import csv
import os
import re
import json
from typing import List, Dict, Union, Optional

harmful_word_phrase_blacklist_data: Optional[List[Dict[str, str]]] = None

def preprocess_text(text: str) -> str:
    """입력 텍스트에서 띄어쓰기 및 문장 부호를 제거합니다."""
    text_without_spaces = re.sub(r'\s', '', text)
    text_without_punctuation = re.sub(r'[.,?!\'"~·—…「」《》〈〉《》\[\]【】()\[\]]', '', text_without_spaces)
    return text_without_punctuation

def load_user_defined_blacklist_from_json(json_file_path: str) -> Union[List[Dict[str, str]], str, None]:
    """
    JSON 파일에서 사용자 정의 유해 단어/구문 블랙리스트를 로드합니다.
    """
    user_defined_blacklist = []
    if not os.path.exists(json_file_path):
        print(f"**[경고]** 사용자 정의 블랙리스트 파일 '{json_file_path}'이 존재하지 않습니다. 기본 블랙리스트만 사용합니다.")
        return None

    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            user_defined_data = json.load(json_file)
            if not isinstance(user_defined_data, list):
                error_message = f"오류 발생: 사용자 정의 블랙리스트 JSON 파일 '{json_file_path}' 형식이 잘못되었습니다. \nJSON 파일은 리스트 형태여야 합니다."
                return error_message
            for item in user_defined_data:
                if not isinstance(item, dict) or "word_phrase" not in item or "category" not in item:
                    error_message = f"오류 발생: 사용자 정의 블랙리스트 JSON 파일 '{json_file_path}' 형식이 잘못되었습니다. \n각 아이템은 'word_phrase'와 'category' 키를 포함하는 딕셔너리 형태여야 합니다."
                    return error_message
                user_defined_blacklist.append({
                    "word_phrase": item["word_phrase"].strip(),
                    "category": item["category"].strip()
                })
        return user_defined_blacklist

    except json.JSONDecodeError as e:
        error_message = f"오류 발생: 사용자 정의 블랙리스트 JSON 파일 '{json_file_path}' 내용 파싱 오류 (JSON 형식이 아님): {e}"
        return error_message
    except Exception as e:
        error_message = f"오류 발생: 사용자 정의 블랙리스트 JSON 파일 '{json_file_path}' 로드 중 오류 발생: {e}"
        return error_message

def load_harmful_word_blacklist_from_csv(csv_file_path: str, user_blacklist_file_path: str = None) -> Union[str, None]:
    """
    CSV 파일에서 유해 단어/구문 블랙리스트를 로드하고, 사용자 정의 블랙리스트와 병합합니다.
    """
    global harmful_word_phrase_blacklist_data
    harmful_word_phrase_blacklist: List[Dict[str, str]] = []
    user_defined_blacklist: List[Dict[str, str]] = []

    if not os.path.exists(csv_file_path):
        error_message = f"오류 발생: 블랙리스트 CSV 파일 경로 '{csv_file_path}'가 존재하지 않습니다."
        return error_message

    try:
        with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
            try:
                csv_reader = csv.DictReader(csvfile)
                header = csv_reader.fieldnames
                if not header or "word_phrase" not in header or "category" not in header:
                    error_message = f"오류 발생: 블랙리스트 CSV 파일 '{csv_file_path}'의 형식이 잘못되었습니다. \n'word_phrase' 및 'category' 컬럼이 CSV 헤더에 포함되어 있어야 합니다."
                    return error_message
                for row in csv_reader:
                    harmful_word_phrase_blacklist.append({
                        "word_phrase": row["word_phrase"].strip(),
                        "category": row["category"].strip()
                    })

                if user_blacklist_file_path:
                    user_defined_blacklist_result = load_user_defined_blacklist_from_json(user_blacklist_file_path)
                    if isinstance(user_defined_blacklist_result, str):
                        return user_defined_blacklist_result
                    elif isinstance(user_defined_blacklist_result, list):
                        user_defined_blacklist = user_defined_blacklist_result
                        print(f"**[성공]** 사용자 정의 블랙리스트 '{user_blacklist_file_path}' 로드 완료. {len(user_defined_blacklist)}개 단어/구문 추가.")
                    else:
                        pass

                merged_blacklist = user_defined_blacklist + harmful_word_phrase_blacklist
                unique_blacklist_word_phrases = set()
                unique_merged_blacklist: List[Dict[str, str]] = []
                for item in merged_blacklist:
                    if item["word_phrase"] not in unique_blacklist_word_phrases:
                        unique_merged_blacklist.append(item)
                        unique_blacklist_word_phrases.add(item["word_phrase"])
                harmful_word_phrase_blacklist_data = unique_merged_blacklist
                return None

            except Exception as e:
                error_message = f"오류 발생: 블랙리스트 CSV 파일 '{csv_file_path}'이 CSV 형식이 아니거나, 내용을 파싱할 수 없습니다."
                return error_message

    except Exception as e:
        print(f"**[디버깅 - 예외 발생]**: 예외가 발생했습니다! - 예외 종류: {type(e)}")
        error_message = f"오류 발생: 블랙리스트 CSV 파일 '{csv_file_path}' 로드 중 오류가 발생했습니다. - {e}"
        print(error_message)
        return error_message

def detect_harmful_words_phrases(text: str, user_blacklist_file_path: str = None) -> Union[List[Dict[str, str]], str, None]:
    """텍스트에서 유해 단어/구문을 탐지합니다."""
    global harmful_word_phrase_blacklist_data
    if harmful_word_phrase_blacklist_data is None: # 블랙리스트 데이터가 None 인 경우, 로드 시도
        blacklist_file_path = "harmful_words_phrases_blacklist.csv"
        error_message = load_harmful_word_blacklist_from_csv(blacklist_file_path, user_blacklist_file_path) # 사용자 정의 블랙리스트 파일 경로 전달
        if error_message: # 블랙리스트 로드 실패 시 오류 메시지 반환
            return error_message
        if harmful_word_phrase_blacklist_data is None: # 여전히 None 인 경우 (로드 실패)
            return None # None 반환 (오류)

    preprocessed_text = preprocess_text(text) # 텍스트 전처리 (띄어쓰기, 문장 부호 제거)
    detected_words_info: List[Dict[str, str]] = [] # 탐지된 단어/구문 정보 리스트 초기화
    for item in harmful_word_phrase_blacklist_data: # 블랙리스트 순회
        word_phrase = item["word_phrase"] # 블랙리스트 단어/구문
        category = item["category"] # 블랙리스트 카테고리
        if word_phrase in preprocessed_text: # 전처리된 텍스트에 블랙리스트 단어/구문 포함 여부 확인
            detected_words_info.append({"word_phrase": word_phrase, "category": category}) # 탐지된 단어/구문 정보 리스트에 추가
    return detected_words_info # 탐지된 단어/구문 정보 리스트 반환

def add_word_to_blacklist(word_phrase: str, category: str) -> str:
    """단어/구문을 블랙리스트에 추가합니다."""
    global harmful_word_phrase_blacklist_data
    if not word_phrase:
        return "오류: 단어/구문을 입력하세요."
    new_item = {"word_phrase": word_phrase, "category": category}
    if harmful_word_phrase_blacklist_data is None:
        harmful_word_phrase_blacklist_data = [new_item]
    else:
        harmful_word_phrase_blacklist_data.append(new_item)
    update_blacklist_in_csv() # CSV 파일 업데이트
    return f"'{word_phrase}' ({category}) 카테고리로 블랙리스트에 추가 완료"

def delete_word_from_blacklist(word_phrase: str) -> str:
    """단어/구문을 블랙리스트에서 삭제합니다."""
    global harmful_word_phrase_blacklist_data
    if not word_phrase:
        return "오류: 삭제할 단어/구문을 입력하세요."
    if harmful_word_phrase_blacklist_data:
        harmful_word_phrase_blacklist_data = [item for item in harmful_word_phrase_blacklist_data if item["word_phrase"] != word_phrase]
        update_blacklist_in_csv() # CSV 파일 업데이트
        return f"'{word_phrase}' 블랙리스트에서 삭제 완료"
    else:
        return "오류: 블랙리스트에 삭제할 단어가 없습니다."

def update_blacklist_in_csv():
    """블랙리스트 데이터를 CSV 파일에 저장합니다."""
    global harmful_word_phrase_blacklist_data
    if harmful_word_phrase_blacklist_data is None:
        return
    blacklist_file_path = "/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv" # Google Drive 경로 명시
    try:
        with open(blacklist_file_path, 'w', encoding='utf-8', newline='') as csvfile:
            fieldnames = ['word_phrase', 'category']
            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            csv_writer.writerows(harmful_word_phrase_blacklist_data)
        print(f"**[성공]** 블랙리스트 CSV 파일 '{blacklist_file_path}' 업데이트 완료")
    except Exception as e:
        error_message = f"**[오류]** 블랙리스트 CSV 파일 '{blacklist_file_path}' 업데이트 실패: {e}"
        print(error_message)

def load_blacklist_data(): # 블랙리스트 로드 함수 (UI 관련 코드 제거)
    global harmful_word_phrase_blacklist_data
    blacklist_file_path = "/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv" # Google Drive 경로 명시
    user_blacklist_file_path = "user_defined_blacklist.json"
    error_message = load_harmful_word_blacklist_from_csv(blacklist_file_path, user_blacklist_file_path)
    if error_message:
        print(f"블랙리스트 로드 실패: {error_message}")
        harmful_word_phrase_blacklist_data = []
    elif harmful_word_phrase_blacklist_data is None:
        print("블랙리스트 로드 실패 (알 수 없는 오류)")
        harmful_word_phrase_blacklist_data = []
    else:
        print("블랙리스트 로드 완료")

# --- [UI 관련 코드 제거] ---
# UI 요소 정의, 이벤트 핸들러, UI 표시 코드 모두 제거됨

# --- [테스트 코드] ---
if __name__ == "__main__": # __name__ == "__main__" 블록 추가 (독립 실행 시에만 테스트 코드 실행)
    load_blacklist_data() # 블랙리스트 로드

    # 텍스트 입력 및 유해 단어/구문 탐지 테스트
    test_text = "이런 멍청이 같은" # 테스트 텍스트
    detected_words = detect_harmful_words_phrases(test_text) # 유해 단어/구문 탐지 함수 호출
    if detected_words: # 탐지된 유해 단어/구문이 있는 경우
        print(f"\n**[탐지 결과]** 텍스트: '{test_text}' 에서 유해 단어/구문이 탐지되었습니다:") # 결과 메시지 출력
        for word in detected_words: # 탐지된 단어/구문 목록 출력
            print(f"- '{word['word_phrase']}' (카테고리: {word['category']})")
    else: # 탐지된 유해 단어/구문이 없는 경우
        print(f"\n**[탐지 결과]** 텍스트: '{test_text}' 에서 유해 단어/구문이 탐지되지 않았습니다.") # 결과 메시지 출력

    # 블랙리스트 단어/구문 추가 테스트
    word_to_add = "test_word" # 추가할 단어/구문
    category_to_add = "일반" # 추가할 카테고리
    add_result_message = add_word_to_blacklist(word_to_add, category_to_add) # 블랙리스트 추가 함수 호출
    print(f"\n**[블랙리스트 추가 결과]**: {add_result_message}") # 추가 결과 메시지 출력

    # 블랙리스트 단어/구문 삭제 테스트
    word_to_delete = "test_word" # 삭제할 단어/구문 (방금 추가한 단어)
    delete_result_message = delete_word_from_blacklist(word_to_delete) # 블랙리스트 삭제 함수 호출
    print(f"\n**[블랙리스트 삭제 결과]**: {delete_result_message}") # 삭제 결과 메시지 출력

    # 블랙리스트 데이터 (harmful_word_phrase_blacklist_data) 내용 확인 (디버깅 용도)
    print(f"\n**[최종 블랙리스트 데이터]**:") # 최종 블랙리스트 데이터 제목 출력
    if harmful_word_phrase_blacklist_data: # 블랙리스트 데이터가 있는 경우
        for item in harmful_word_phrase_blacklist_data: # 블랙리스트 아이템 순회하며 출력
            print(f"- '{item['word_phrase']}' (카테고리: {item['category']})") # 각 아이템 정보 출력
    else: # 블랙리스트 데이터가 없는 경우
        print("블랙리스트 데이터가 없습니다.") # 블랙리스트 없음 메시지 출력

**[경고]** 사용자 정의 블랙리스트 파일 'user_defined_blacklist.json'이 존재하지 않습니다. 기본 블랙리스트만 사용합니다.
블랙리스트 로드 완료

**[탐지 결과]** 텍스트: '이런 멍청이 같은' 에서 유해 단어/구문이 탐지되었습니다:
- '멍청이' (카테고리: 욕설)
**[성공]** 블랙리스트 CSV 파일 '/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv' 업데이트 완료

**[블랙리스트 추가 결과]**: 'test_word' (일반) 카테고리로 블랙리스트에 추가 완료
**[성공]** 블랙리스트 CSV 파일 '/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv' 업데이트 완료

**[블랙리스트 삭제 결과]**: 'test_word' 블랙리스트에서 삭제 완료

**[최종 블랙리스트 데이터]**:
- '멍청이' (카테고리: 욕설)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 필요한 파일 경로 설정
blacklist_file_path = "/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv"

Mounted at /content/drive


In [ ]:
# AI API 연결 코드 (저를 통해 제공받은 코드 붙여넣기)

In [ ]:
# 테스트 함수 실행

In [ ]:
import csv
from google.colab import drive
import os

# Google Drive 연결 (이미 연결되어 있다면 이 부분은 건너뛰어도 됩니다.)
drive.mount('/content/drive', force_remount=True)

def add_word_to_blacklist_csv(file_path: str, word_phrase: str, category: str):
    """주어진 CSV 파일에 단어/구문을 블랙리스트로 추가합니다."""
    try:
        # 파일이 존재하지 않으면 헤더를 포함하여 새로 생성
        if not os.path.exists(file_path):
            with open(file_path, 'w', encoding='utf-8', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['word_phrase', 'category'])
                writer.writerow([word_phrase, category])
            print(f"**[성공]** 블랙리스트 파일 '{file_path}' 생성 및 '{word_phrase}' 추가 완료")
        else:
            # 파일이 존재하면 추가 모드로 열어 단어 추가
            with open(file_path, 'a', encoding='utf-8', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([word_phrase, category])
            print(f"**[성공]** 블랙리스트 파일 '{file_path}'에 '{word_phrase}' 추가 완료")
    except Exception as e:
        error_message = f"**[오류]** 블랙리스트 파일 '{file_path}'에 '{word_phrase}' 추가 실패: {e}"
        print(error_message)

# 추가하고 싶은 단어와 카테고리 설정
word_to_add = "멍청이"
category_to_add = "욕설"

# 블랙리스트 파일 경로 (본인의 Google Drive 경로에 맞춰 수정해주세요)
blacklist_file_path = "/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv"

# 함수 호출하여 단어 추가
add_word_to_blacklist_csv(blacklist_file_path, word_to_add, category_to_add)

# (선택 사항) 다른 단어 추가 예시
# add_word_to_blacklist_csv(blacklist_file_path, "바보", "비속어")

Mounted at /content/drive
**[성공]** 블랙리스트 파일 '/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv' 생성 및 '멍청이' 추가 완료


In [ ]:
!python ai_interface.py

사용자: 안녕하세요?
AI 응답: 안녕하세요! 무엇을 도와드릴까요? 😊



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from ai_interface import send_message_to_ai
from main import detect_harmful_words_phrases # 유해 단어 탐지 함수 불러오기

def start_chat():
    print("AI 채팅을 시작합니다. '종료'를 입력하면 채팅이 종료됩니다.")
    while True:
        user_input = input("사용자: ")
        if user_input.lower() == '종료':
            print("채팅을 종료합니다.")
            break

        # 사용자 입력에 대한 유해 단어 필터링
        detected_harmful_words_user = detect_harmful_words_phrases(user_input)
        if detected_harmful_words_user:
            print(f"**[경고]** 입력에 유해 단어/구문이 포함되어 있습니다: {detected_harmful_words_user}")
            continue # 유해 단어가 있으면 AI에게 보내지 않고 다시 입력 받음

        ai_response = send_message_to_ai(user_input)

        # AI 응답에 대한 유해 단어 필터링
        detected_harmful_words_ai = detect_harmful_words_phrases(ai_response)
        if detected_harmful_words_ai:
            print(f"**[경고]** AI 응답에 유해 단어/구문이 포함되어 있습니다: {detected_harmful_words_ai}")
            print("**[주의]** AI 응답에서 유해 단어가 감지되어 표시하지 않습니다.")
            ai_response = "[유해 단어 감지됨]" # 또는 다른 대체 메시지 처리
        else:
            print(f"AI 응답: {ai_response}")

if __name__ == "__main__":
    start_chat()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


SyntaxError: invalid syntax (main.py, line 4)

In [ ]:
!python main.py

  File "/content/main.py", line 4
    detected_harmful =
                      ^
SyntaxError: invalid syntax


In [ ]:
import csv
import os
import re
import json
from typing import List, Dict, Optional
from google.colab import drive

# --- Google Drive 마운트 및 파일 처리 ---

def mount_google_drive():
    """구글 드라이브를 Colab 환경에 마운트"""
    drive.mount('/content/drive')  # /content/drive 경로로 구글 드라이브가 마운트됨

# --- 블랙리스트 로딩 및 관리 함수 ---

class BlacklistLoader:
    def __init__(self, csv_path: str, json_path: Optional[str] = None):
        """CSV 파일 경로, JSON 파일 경로를 받아 블랙리스트를 초기화"""
        self.csv_path = csv_path
        self.json_path = json_path
        self.blacklist_data: List[Dict[str, str]] = []
        self.load_blacklist()

    def load_blacklist(self):
        """블랙리스트 로딩: CSV 및 JSON 파일에서 불러오기"""
        csv_data = self._load_csv()
        json_data = self._load_json() if self.json_path else []
        # CSV와 JSON 데이터를 병합하여 최종 블랙리스트 데이터 생성
        self.blacklist_data = self._merge_blacklists(csv_data, json_data)

    def _load_csv(self) -> List[Dict[str, str]]:
        """CSV 파일에서 블랙리스트 로드"""
        if not os.path.exists(self.csv_path):
            print(f"[경고] 블랙리스트 CSV 파일이 존재하지 않습니다: {self.csv_path}")
            return []

        try:
            with open(self.csv_path, 'r', encoding='utf-8') as csvfile:
                csv_reader = csv.DictReader(csvfile)
                if "word_phrase" not in csv_reader.fieldnames or "category" not in csv_reader.fieldnames:
                    print(f"[오류] CSV 형식이 잘못되었습니다: {self.csv_path}")
                    return []
                return [
                    {"word_phrase": row["word_phrase"].strip(), "category": row["category"].strip()}
                    for row in csv_reader
                ]
        except Exception as e:
            print(f"[오류] CSV 로드 중 오류 발생: {e}")
            return []

    def _load_json(self) -> List[Dict[str, str]]:
        """JSON 파일에서 사용자 정의 블랙리스트 로드"""
        if not os.path.exists(self.json_path):
            print(f"[경고] 사용자 정의 블랙리스트 파일이 없습니다: {self.json_path}")
            return []

        try:
            with open(self.json_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                if not isinstance(data, list):
                    print(f"[오류] JSON 형식이 잘못되었습니다: {self.json_path}")
                    return []
                return [
                    {"word_phrase": item["word_phrase"].strip(), "category": item["category"].strip()}
                    for item in data if isinstance(item, dict) and "word_phrase" in item and "category" in item
                ]
        except json.JSONDecodeError as e:
            print(f"[오류] JSON 파싱 오류: {e}")
            return []
        except Exception as e:
            print(f"[오류] JSON 로드 중 오류 발생: {e}")
            return []

    def _merge_blacklists(self, csv_data: List[Dict[str, str]], json_data: List[Dict[str, str]]) -> List[Dict[str, str]]:
        """CSV 및 JSON 데이터를 병합하고 중복 제거"""
        unique_entries = {item["word_phrase"]: item for item in csv_data + json_data}
        return list(unique_entries.values())

    def save_blacklist(self):
        """블랙리스트를 CSV 파일에 저장"""
        try:
            with open(self.csv_path, 'w', encoding='utf-8', newline='') as csvfile:
                fieldnames = ['word_phrase', 'category']
                csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                csv_writer.writeheader()
                csv_writer.writerows(self.blacklist_data)
            print(f"[성공] 블랙리스트 저장 완료: {self.csv_path}")
        except Exception as e:
            print(f"[오류] 블랙리스트 저장 실패: {e}")

    def add_to_blacklist(self, word_phrase: str, category: str):
        """블랙리스트에 항목 추가"""
        if any(item["word_phrase"] == word_phrase for item in self.blacklist_data):
            print(f"[정보] '{word_phrase}' 이미 블랙리스트에 존재함")
            return
        self.blacklist_data.append({"word_phrase": word_phrase, "category": category})
        self.save_blacklist()

    def remove_from_blacklist(self, word_phrase: str):
        """블랙리스트에서 항목 제거"""
        self.blacklist_data = [item for item in self.blacklist_data if item["word_phrase"] != word_phrase]
        self.save_blacklist()

# --- 텍스트 전처리 및 유해 단어 탐지 함수 ---

def preprocess_text(text: str) -> str:
    """입력 텍스트에서 띄어쓰기 및 문장 부호를 제거"""
    text_without_spaces = re.sub(r'\s', '', text)
    return re.sub(r'[.,?!\'"~·—…「」《》〈〉《》\[\]【】()\[\]]', '', text_without_spaces)

def detect_harmful_words(text: str, blacklist_loader: BlacklistLoader) -> List[Dict[str, str]]:
    """텍스트에서 유해 단어/구문을 탐지"""
    preprocessed_text = preprocess_text(text)
    return [
        {"word_phrase": item["word_phrase"], "category": item["category"]}
        for item in blacklist_loader.blacklist_data if item["word_phrase"] in preprocessed_text
    ]

# --- 테스트 실행 ---
if __name__ == "__main__":
    # 구글 드라이브 마운트
    mount_google_drive()

    # 블랙리스트 로딩 (경로 수정)
    blacklist_loader = BlacklistLoader(
        csv_path="/content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv",
        json_path="/content/drive/MyDrive/AI_Vtuber/user_defined_blacklist.json"
    )

    # 텍스트에서 유해 단어 탐지
    test_text = "이런 멍청이 같은"
    detected_words = detect_harmful_words(test_text, blacklist_loader)

    print("\n[탐지 결과]")
    if detected_words:
        for word in detected_words:
            print(f"- '{word['word_phrase']}' (카테고리: {word['category']})")
    else:
        print("유해 단어/구문이 발견되지 않았습니다.")

    # 블랙리스트 관리 예시
    blacklist_loader.add_to_blacklist("새로운금지어", "일반")
    blacklist_loader.remove_from_blacklist("새로운금지어")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[경고] 사용자 정의 블랙리스트 파일이 없습니다: /content/drive/MyDrive/AI_Vtuber/user_defined_blacklist.json

[탐지 결과]
- '멍청이' (카테고리: 욕설)
[성공] 블랙리스트 저장 완료: /content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv
[성공] 블랙리스트 저장 완료: /content/drive/MyDrive/AI_Vtuber/harmful_words_phrases_blacklist.csv
